In [1]:
import sys
sys.path.append( '..' )
from utilities.get_data import get_historical_from_path
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta

In [2]:
mypath = "../database/quick_analysis"
file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(mypath+"/"+file)
    
print("All data loaded")

All data loaded


In [3]:
df_list["BTC"]

,open,high,low,close,volume
date,,,,,
2022-02-10 00:00:00,44372.71,44400.00,44030.10,44103.29,1611.55153
2022-02-10 01:00:00,44103.29,44261.25,44028.57,44253.22,1054.58247
2022-02-10 02:00:00,44253.23,44360.00,43800.00,43800.01,1402.61623
2022-02-10 03:00:00,43800.00,43858.41,43542.14,43775.42,1288.67942
2022-02-10 04:00:00,43775.42,43911.57,43575.22,43783.82,942.91797
...,...,...,...,...,...
2022-03-23 11:00:00,42156.52,42186.07,41878.82,42039.78,1361.04251
2022-03-23 12:00:00,42039.77,42438.71,41881.00,42359.04,2383.91223
2022-03-23 13:00:00,42359.04,42450.00,41976.87,42020.98,2587.90169


In [5]:
def get_analyisis_from_window(window=24):
    metric_list = {}
    for coin in df_list:
        try:
            df = df_list[coin]
            df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)
            df["rvolume"] = df["volume"] * df["close"]
            df["olhc"] = (df["open"] + df["low"] + df["high"] + df["close"])/4
            df["atr"] = (ta.volatility.average_true_range(high=df['high'], low=df['low'], close=df['close'])/df["close"])*100
            # df["norm"].dropna(inplace=True) 
            volatility = df["atr"].mean()
            mean_volume = df["rvolume"].mean()
            std = df["olhc"].std()/df["olhc"].mean()
            perf = (df["close"].iloc[-1] - df["close"].iloc[0]) / df["close"].iloc[0]
            return_vs_vol = perf/volatility
            last_volume = df["rvolume"].iloc[-window:].mean()
            volume_evolution = last_volume / mean_volume
            last_volatility = df.iloc[-window:]["olhc"].std() / df.iloc[-window:]["olhc"].mean()
            last_perf = (df.iloc[-1]["close"] - df.iloc[-window]["close"]) / df.iloc[-window]["close"]
            volatility_evolution = last_volatility / volatility
            last_return_vs_vol = last_perf / last_volatility
            metric_list[coin] = {
                "mean_volume": round(mean_volume), 
                "last_volume": round(last_volume), 
                "volume_evolution": round(volume_evolution,2),
                "mean_volatility": round(volatility,2), 
                "last_volatility": round(last_volatility*100,2),
                "volatility_evolution": round(volatility_evolution*100,2), 
                "return": round(perf*100,2), 
                "last_return":round(last_perf*100,2),
                "return_vs_vol": round(return_vs_vol*100,2),        
                "last_return_vs_vol":round(last_return_vs_vol,2)
                }
        except:
            # print("Error on", coin)
            # raise 
            pass

    return pd.DataFrame.from_dict(metric_list, orient='index')
    

In [9]:
df_metric = get_analyisis_from_window(12).sort_values(by="last_return", ascending=False)
df_metric.iloc[:30]

,mean_volume,last_volume,volume_evolution,mean_volatility,last_volatility,volatility_evolution,return,last_return,return_vs_vol,last_return_vs_vol
LRC,1469399,26694177,18.17,1.90,14.14,7.43,2.53,32.78,1.33,2.32
CHR,742344,1468713,1.98,2.02,4.93,2.44,-24.41,15.47,-12.11,3.14
EPS,324864,576932,1.78,2.16,3.90,1.80,-4.06,14.29,-1.88,3.67
LINA,254809,1210815,4.75,1.79,5.04,2.82,-18.25,14.19,-10.22,2.81
JASMY,3967230,16321017,4.11,3.91,4.34,1.11,11.41,12.25,2.92,2.82
APE,38787276,21955844,0.57,6.28,2.98,0.47,48.18,11.91,7.67,3.99
QI,286998,460948,1.61,2.51,2.84,1.13,-19.58,11.59,-7.81,4.09
LOKA,695043,1314985,1.89,2.92,3.95,1.35,15.63,11.27,5.35,2.85
REQ,190448,807908,4.24,2.01,4.21,2.10,-15.45,11.13,-7.69,2.64
HIGH,518765,761008,1.47,2.78,2.28,0.82,3.77,11.05,1.35,4.86
